In [9]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [ ]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

if "cn-" in region_name:
    with open('./code/requirements.txt', 'r') as original: data = original.read()
    with open('./code/requirements.txt', 'w') as modified: modified.write("-i https://pypi.tuna.tsinghua.edu.cn/simple\n" + data)

!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'llm_falcon-7B')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'llm_falcon-7B')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

model_name = None
entry_point = 'inference.py'
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = './code',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

endpoint_name = 'pytorch-inference-llm-v1'
instance_type = 'ml.g5.4xlarge'
instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

arn:aws:iam::150333911459:role/service-role/AmazonSageMaker-ExecutionRole-20220802T133420
sagemaker-us-east-1-150333911459
dummy
upload: ./model.tar.gz to s3://sagemaker-us-east-1-150333911459/llm_falcon-7B/assets/model.tar.gz
-----------

In [11]:
inputs= {
    "ask": "你好!"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "晚上睡不着应该怎么办"

}

response = predictor.predict(inputs)
print(response["answer"])

你好!

晚上睡不着应该怎么办？

如果你晚上睡不着，可以尝试以下方法：

1. 放松身体：可以进行一些放松身体的活动，如瑜伽、冥想、深呼吸等。

2. 调整环境：保持房间安静、舒适，并调整好温度和光线。

3. 避免使用电子设备：在睡觉前避免使用电子设备，如手机、平板电脑等。

4. 改变睡姿：尝试改变睡姿，如侧卧、仰卧等。

5. 喝一杯温牛奶：温牛奶中含有色氨酸，可以帮助入睡。

6. 避免过度兴奋：晚上避免过度兴奋，如剧烈运动、看刺激的电影等。

如果以上方法都不能帮助你入睡，建议你去看医生，以了解是否存在其他健康问题。



In [22]:
inputs= {
    "ask": """
你是一个程序员，请根据以下内容写一段代码

（苹果是红颜色，价格是5元，香蕉是黄颜色，价格是3元。）

请写一段可直接执行的python代码，可以计算n个苹果加上m个香蕉后总共的价格，并输出结果。
"""

}

response = predictor.predict(inputs)
print(response["answer"])


你是一个程序员，请根据以下内容写一段代码

（苹果是红颜色，价格是5元，香蕉是黄颜色，价格是3元。）

请写一段可直接执行的python代码，可以计算n个苹果加上m个香蕉后总共的价格，并输出结果。

示例输出：

总价：20

代码如下：

```python
n = int(input("请输入苹果的数量："))
m = int(input("请输入香蕉的数量："))

total_price = n * 5 + m * 3

print("总价：", total_price)
```

